#### 20210112

> Is that true?

> 高回报的投资： 
1. 读书，去别人的灵魂里偷窥；
2. 旅行，去陌生的环境里去感悟；
3. 电影，去荧屏里感受别人的生活历程；
4. 冥想，去自己内心的秘境里探寻。

---
```
思考，使用简单工具，排列组合来解决复杂的问题。

1、网络上现成的资料
　　格式: sed -i "s/查找字段/替换字段/g" `grep 查找字段 -rl 路径`

　　linux sed 批量替换多个文件中的字符串

　　sed -i "s/oldstring/newstring/g" `grep oldstring -rl yourdir`

　　例如：替换/home下所有文件中的www.admin99.net为admin99.net

　　sed -i "s/www.admin99.net/admin99.net/g" `grep www.admin99.net -rl /home`

　　exp:sed -i "s/shabi/$/g" `grep shabi -rl ./`

2、自己额外附加

　　2.1 将文件1.txt内的文字“garden”替换成“mirGarden”

　　# sed -i "s/garden/mirGarden/g" 1.txt //sed -i 很简单

　　2.2 将当前目录下的所有文件内的“garden”替换成“mirGarden”

　　## sed -i "s/garden/mirGarden/g" `ls` //其实也就是ls出多个文件名而已
```

